In [1]:
import torch
import torch.onnx
!pip install onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.0 MB/s eta 0:00:00


In [2]:
# Создание модели GRU
model = torch.nn.GRU(10, 20, 2)
dummy_input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = model(dummy_input, h0)
print(output)

tensor([[[ 4.4246e-01, -5.4133e-02,  1.0080e-01, -1.9254e-02,  1.7461e-01,
          -7.2246e-02,  1.3074e-01,  1.7277e-03, -1.1351e+00, -1.2811e+00,
           3.1864e-01,  1.2165e+00,  1.5116e-01,  7.7083e-01, -1.5962e+00,
          -1.1919e+00, -7.9727e-01,  5.3192e-02,  3.5041e-01,  2.8165e-01],
         [-1.6802e-02, -4.8816e-01,  9.2177e-01, -6.0001e-01, -9.9407e-01,
           7.4527e-01,  2.4008e-01,  9.1043e-01,  1.4454e-02, -2.4006e-01,
          -2.2740e-01,  5.8026e-02, -1.3282e-01,  2.2173e-01, -8.6415e-01,
           4.5727e-01,  1.6189e-01,  7.8419e-02, -4.0676e-01, -1.1540e-01],
         [ 2.0793e-01, -1.2998e+00, -1.0992e+00,  2.5263e-01,  2.1871e-01,
           1.0723e+00,  6.7791e-01,  1.3142e+00, -4.1194e-01,  2.2351e-01,
          -2.0187e-01,  5.7661e-01,  4.5886e-01, -2.4328e-02,  1.3183e-01,
          -5.2602e-01, -7.4659e-01,  1.9287e-01, -4.9474e-01, -3.9465e-01]],

        [[ 6.2006e-01, -7.1234e-02,  1.4415e-02,  2.2362e-01,  6.3175e-02,
           1.5389e-0

In [3]:
# Конвертация в ONNX
torch.onnx.export(model, dummy_input, "gru.onnx")

/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_opset9.py:4661: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [4]:
# Загрузка и запуск модели в ONNX
import onnx
import onnxruntime
onnx_model = onnx.load("gru.onnx")
ort_session = onnxruntime.InferenceSession("gru.onnx")
ort_inputs = {ort_session.get_inputs()[0].name: dummy_input.numpy()}
ort_outs = ort_session.run(None, ort_inputs)
print(ort_outs)

[array([[[ 2.96349935e-02,  4.33636121e-02,  3.08799352e-02,
          9.67308432e-02, -1.22858363e-03, -1.34463897e-02,
          1.22010410e-01, -1.57345876e-01,  6.85085263e-03,
         -2.22928729e-02,  1.80333465e-01,  2.40926701e-03,
          1.29596777e-02, -1.31591842e-01, -1.41851325e-02,
          1.12930024e-02, -7.80541589e-03, -6.78019449e-02,
         -8.54326338e-02, -5.19844517e-02],
        [-3.37405223e-03,  5.18026985e-02,  2.32792404e-02,
         -7.81426504e-02, -1.48854628e-02, -2.59167310e-02,
          1.67888418e-01, -1.45626500e-01, -2.67681889e-02,
          1.59231760e-03,  5.22918887e-02,  2.46134829e-02,
          4.68178429e-02, -1.11663327e-01, -8.57776613e-04,
          6.48833811e-02, -1.04568936e-02,  2.81912908e-02,
         -1.17346250e-01, -6.68459460e-02],
        [ 1.16380908e-01, -1.29014570e-02,  6.29655644e-02,
          1.12283021e-01,  5.29363975e-02,  1.15377391e-02,
          1.31040588e-01, -1.92302465e-01,  4.42420729e-02,
         -9

In [5]:
# Обратная конвертация
!pip install onnx2torch
from onnx2torch.converter import convert
torch_model = convert("gru.onnx")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

NotImplementedError: Converter is not implemented (OperationDescription(domain='', operation_type='GRU', version=14))